# US States Energy Data

This dataset contains all energy data of the US and its states and territories.
The dataset is sourced from EIA's State Energy Data System, 1960-2019.

We used a consolidated data file of more than 1.8 million records that is available on EIA's SEDS [seds-data-complete website](https://www.eia.gov/state/seds/seds-data-complete.php?sid=US). The calculation of the algorithms of the variables is done by reviewing EIA's SEDS [Codes and Documentation](https://www.eia.gov/state/seds/seds-data-complete.php?sid=US) (download the xlsx file [here](https://www.eia.gov/state/seds/CDF/Codes_and_Descriptions.xlsx)), [Technical Notes and Documentation](https://www.eia.gov/state/seds/seds-technical-notes-complete.php?sid=US), and [Data and methodology changes](https://www.eia.gov/state/seds/seds-data-changes.php?sid=US)

The variables and documentation for this dataset can be found in the `info.xlsx` Excel workbook in the `info` directory. In the `info.xlsx` Excel workbook, the `calc` sheet contains a table of how to calculate the variables.

**Note**: For this notebook, We used the file that contains records from 1960-2019. The file can be downloaded [here](https://www.eia.gov/state/seds/CDF/Complete_SEDS.csv) 

There is an updated file that contains the data from 1960-2020, available on EIA's SEDS updated [website](https://www.eia.gov/state/seds/seds-data-fuel.php?sid=US) website. The file can be downloaded using this [link](https://www.eia.gov/state/seds/sep_update/Complete_SEDS_update.csv).

For the Documentations' 2020 update, the new Technical Notes and Documentation is available on [this website](https://www.eia.gov/state/seds/seds-technical-notes-updates.php?sid=US), and the new Codes and Description is available [here](https://www.eia.gov/state/seds/seds-data-complete.php?sid=US) (download the xlsx file [here](https://www.eia.gov/state/seds/CDF/Codes_and_Descriptions.xlsx))


## Data Dictionary (Info)

Update: June 10th, 2022. 10:00pm

This section will evaluate the transition of info from Excel worksheets to usable files in an application.

The calculation of the algorithms of the variables is done by reviewing EIA's SEDS [Codes and Documentation](https://www.eia.gov/state/seds/seds-data-complete.php?sid=US) (download the xlsx file [here](https://www.eia.gov/state/seds/CDF/Codes_and_Descriptions.xlsx)), [Technical Notes and Documentation](https://www.eia.gov/state/seds/seds-technical-notes-complete.php?sid=US), and [Data and methodology changes](https://www.eia.gov/state/seds/seds-data-changes.php?sid=US)

For the 2020 update, the new Technical Notes and Documentation is available on [this website](https://www.eia.gov/state/seds/seds-technical-notes-updates.php?sid=US), and the new Codes and Description is available [here](https://www.eia.gov/state/seds/seds-data-complete.php?sid=US) (download the xlsx file [here](https://www.eia.gov/state/seds/CDF/Codes_and_Descriptions.xlsx))


In [1]:
import pandas as pd

### Load info and calc from the Excel book 

`df_info` is the table containing all information of all neeeded variables of the dataset.

`df_calc` is the table that list the calculation method of each variable.

`df_groups` is the table that list all groups and subgroups (parents and childs)

In [2]:
# read files
# load info and calc from Excel book

info_directory = "info/"

df = pd.read_excel(info_directory + "info.xlsx", sheet_name=['info', 'calc'])
df_info = df["info"]
df_calc = df["calc"]

# set df_info index as id
df_info.set_index("id", drop=False, inplace=True)
df_info

,id,name,type,order,column,color,source,target
id,,,,,,,,
WoodProduction,WoodProduction,Wood Production,node,0.0,1.0,grey,NaN,NaN
Solar,Solar,Solar,node,1.0,2.0,yellow,NaN,NaN
Wind,Wind,Wind,node,2.0,2.0,lightblue,NaN,NaN
Geothermal,Geothermal,Geothermal,node,3.0,2.0,brown,NaN,NaN
Hydropower,Hydropower,Hydropower,node,4.0,2.0,blue,NaN,NaN
...,...,...,...,...,...,...,...,...
Wood->Commercial,Wood->Commercial,NaN,link,NaN,NaN,NaN,Wood,Commercial
WoodProduction->Wood,WoodProduction->Wood,NaN,link,NaN,NaN,NaN,WoodProduction,Wood
WoodProduction->DensifiedBiomassExport,WoodProduction->DensifiedBiomassExport,NaN,link,NaN,NaN,NaN,WoodProduction,DensifiedBiomassExport


### Load groups table

In [3]:
# retrieve df_groups from df_calc
# (parents and childs)

# list of groups
list_groups = df_info[df_info["type"] == "group"]["id"].tolist()

# df_groups
df_groups = df_calc[df_calc["id"].isin(list_groups)]
df_groups

del list_groups # remove not needed variables of this script

### Calculate groups leafs

Identify the leafs of each group. Each group is the sum of a set of subgroups.

For example:
```py
Total = Renewables + FossilFuel + Nuclear
```

We need to calculate each group's set of leafs (nodes), for easier data manipulations in the visualisation, especially the links.

Our strategy is as follows: If all subgroups of a group already had their leafs calculated (or are leafs themselves), then the leafs of the group is the sum of all leafs of the subgroups. The order of calculations will be from the leafs, to the parents in low level, to the parents in higher levels, and so on.

In [4]:
# create a pd.Series dictionary that records a list of leafs for each group
# except for nodes, each node's value in this Series equals a list of itself (node: [node])
# this is used for easier list additions

# list of all nodes
list_nodes= df_info[df_info["type"].isin(["node"])]['id'].tolist()
list_nodes

# series of leafs
sf_leafs = pd.Series({i:[i] for i in list_nodes})
sf_leafs

WoodProduction                    [WoodProduction]
Solar                                      [Solar]
Wind                                        [Wind]
Geothermal                            [Geothermal]
Hydropower                            [Hydropower]
Wood                                        [Wood]
Waste                                      [Waste]
Biodiesel                              [Biodiesel]
BiodieselLoss                      [BiodieselLoss]
FuelEthanol                          [FuelEthanol]
FuelEthanolLoss                  [FuelEthanolLoss]
Nuclear                                  [Nuclear]
Coal                                        [Coal]
NaturalGas                            [NaturalGas]
Petroleum                              [Petroleum]
ElectricPower                      [ElectricPower]
Transportation                    [Transportation]
Industrial                            [Industrial]
Commercial                            [Commercial]
Residential                    

In [5]:
# create a Series that records the list of subgroups (childs) of each group (parent)
sf_hierarchy = df_groups.groupby("id").apply(lambda x: x["Variable"].tolist())
sf_hierarchy

id
BiodieselSum                                      [Biodiesel, BiodieselLoss]
Biofuel                                       [BiodieselSum, FuelEthanolSum]
Biomass                                                 [Biofuel, WoodWaste]
Clean                                                   [Nuclear, Renewable]
Consumption                [ElectricPower, Transportation, Industrial, Co...
FossilFuel                                     [Coal, NaturalGas, Petroleum]
FuelEthanolSum                                [FuelEthanol, FuelEthanolLoss]
NoncombustibleRenewable                [Solar, Wind, Geothermal, Hydropower]
Nonrenewable                                           [FossilFuel, Nuclear]
Renewable                                 [NoncombustibleRenewable, Biomass]
Total                                              [Nonrenewable, Renewable]
WoodWaste                                                      [Wood, Waste]
dtype: object

In [6]:
# calculate the leafs of each group

# Our strategy is as follows:
# If all subgroups of a group already had their leafs calculated (or are leafs themselves), then the leafs of the group is the sum of all leafs of the subgroups
# The order of calculations will be from the leafs, to the parents in low level, to the parents in higher levels, and so on

# number of groups still needed to be identified the leafs
# by default, it is the total number of groups
to_calculate = len(sf_hierarchy)

while(to_calculate > 0):
    
    # a dictionary of all groups that are eligible to be calculated (have all subgroups/leafs calculated)
    d_to_calculate = {}

    # search for all groups that are eligible to be calculated
    for index, value in sf_hierarchy.iteritems():

        # skip if the group has already been calculated
        if(index in sf_leafs.index): 
            sf_hierarchy.drop(index)
            continue

        # if all subgroups have been calculated, but the group is not calculated, then add the group to the to be calculated dictionary
        if(all(v in sf_leafs.index for v in value)): 
            d_to_calculate[index] = value

    # check how many groups have just been calculated. if 0 (no more groups needed to calculate), then break. 
    to_calculate = len(d_to_calculate)

    if(to_calculate == 0): break

    # calculate the leafs of each group
    for index, value in d_to_calculate.items():

        # initialize list
        sf_leafs[index] = []

        # aggregate the set of leafs of each subgroup of the group
        for v in value:
            sf_leafs[index] += sf_leafs[v]


sf_leafs


WoodProduction                                              [WoodProduction]
Solar                                                                [Solar]
Wind                                                                  [Wind]
Geothermal                                                      [Geothermal]
Hydropower                                                      [Hydropower]
Wood                                                                  [Wood]
Waste                                                                [Waste]
Biodiesel                                                        [Biodiesel]
BiodieselLoss                                                [BiodieselLoss]
FuelEthanol                                                    [FuelEthanol]
FuelEthanolLoss                                            [FuelEthanolLoss]
Nuclear                                                            [Nuclear]
Coal                                                                  [Coal]

In [7]:
# remove nodes from sf_leafs
# the nodes only serves as a method for calculations
sf_leafs = sf_leafs.drop(labels=list_nodes)
sf_leafs

BiodieselSum                                      [Biodiesel, BiodieselLoss]
Consumption                [ElectricPower, Transportation, Industrial, Co...
FossilFuel                                     [Coal, NaturalGas, Petroleum]
FuelEthanolSum                                [FuelEthanol, FuelEthanolLoss]
NoncombustibleRenewable                [Solar, Wind, Geothermal, Hydropower]
WoodWaste                                                      [Wood, Waste]
Biofuel                    [Biodiesel, BiodieselLoss, FuelEthanol, FuelEt...
Nonrenewable                          [Coal, NaturalGas, Petroleum, Nuclear]
Biomass                    [Biodiesel, BiodieselLoss, FuelEthanol, FuelEt...
Renewable                  [Solar, Wind, Geothermal, Hydropower, Biodiese...
Clean                      [Nuclear, Solar, Wind, Geothermal, Hydropower,...
Total                      [Coal, NaturalGas, Petroleum, Nuclear, Solar, ...
dtype: object

### Merge the calculated leafs data to info

In [8]:
# merge sf_leafs to df_info
df_info = pd.merge(df_info, sf_leafs.to_frame("leafs"), how="left", left_index=True, right_index=True)
df_info

,id,name,type,order,column,color,source,target,leafs
id,,,,,,,,,
WoodProduction,WoodProduction,Wood Production,node,0.0,1.0,grey,NaN,NaN,NaN
Solar,Solar,Solar,node,1.0,2.0,yellow,NaN,NaN,NaN
Wind,Wind,Wind,node,2.0,2.0,lightblue,NaN,NaN,NaN
Geothermal,Geothermal,Geothermal,node,3.0,2.0,brown,NaN,NaN,NaN
Hydropower,Hydropower,Hydropower,node,4.0,2.0,blue,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
Wood->Commercial,Wood->Commercial,NaN,link,NaN,NaN,NaN,Wood,Commercial,NaN
WoodProduction->Wood,WoodProduction->Wood,NaN,link,NaN,NaN,NaN,WoodProduction,Wood,NaN
WoodProduction->DensifiedBiomassExport,WoodProduction->DensifiedBiomassExport,NaN,link,NaN,NaN,NaN,WoodProduction,DensifiedBiomassExport,NaN


### Write info to files

In [9]:
# write df_info to json
# remove every key with nan values before writing to json
df_info.agg(lambda x: x.dropna().to_dict(), axis=1).to_json("info.json")

In [10]:
# write df_info to csv
df_info.to_csv(info_directory + "info.csv", index=False)

## Dataset

Update: June 10th, 2022. 10:00 pm

This section will elaborate on the data processing process.

We used a consolidated data file of more than 1.8 million records that is available on EIA's SEDS [seds-data-complete website](https://www.eia.gov/state/seds/seds-data-complete.php?sid=US).

**Note**: For this notebook, We used the file that contains records from 1960-2019. The file can be downloaded [here](https://www.eia.gov/state/seds/CDF/Complete_SEDS.csv) 

There is an updated file that contains the data from 1960-2020, available on EIA's SEDS updated [website](https://www.eia.gov/state/seds/seds-data-fuel.php?sid=US) website. The file can be downloaded using this [link](https://www.eia.gov/state/seds/sep_update/Complete_SEDS_update.csv).

In [11]:
# import needed packages
import pandas as pd

### Read and Clean the dataset

In [12]:
# read the dataset dictionary (info, calc)
df = pd.read_excel("info/info.xlsx", sheet_name=['info', 'calc'])
df_info = df["info"]
df_calc = df["calc"]

In [13]:
# read the source dataset
df_data = pd.read_csv("https://www.eia.gov/state/seds/CDF/Complete_SEDS.csv")
df_data

# drop Data_Status
# we don't need it since Data_Status all equals 2019F
# to verify it, split this cell in this line here and run 
# ```
# df_data["Data_Status"].drop_duplicates()
# ```
df_data = df_data.drop(columns="Data_Status")
df_data

# select only needed MSN
df_data = df_data[df_data["MSN"].isin(df_calc["Variable"].to_list())]
df_data

# pivot the dataset
df_data = pd.pivot(df_data, index=["StateCode", "Year"], columns="MSN", values="Data")
df_data

# reset index so that the pivoted dataset becomes a normal table again
df_data.reset_index(inplace=True)
df_data


MSN,StateCode,Year,BDACB,BDLCB,BDTCB,CCEXB,CCIMB,CLACB,CLCCB,CLEIB,...,WDRCB,WDTCB,WSCCB,WSEIB,WSICB,WSTCB,WYCCB,WYEGB,WYICB,WYTCB
0,AK,1960,0.0,0.0,0.0,NaN,NaN,86.0,496.0,914.0,...,1806.0,3681.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,AK,1961,0.0,0.0,0.0,NaN,NaN,42.0,496.0,1127.0,...,1823.0,4145.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,AK,1962,0.0,0.0,0.0,NaN,NaN,43.0,593.0,1373.0,...,1703.0,4246.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,AK,1963,0.0,0.0,0.0,NaN,NaN,36.0,393.0,1482.0,...,1651.0,4383.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,AK,1964,0.0,0.0,0.0,NaN,NaN,33.0,306.0,2279.0,...,1703.0,4728.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3115,WY,2015,213.0,0.0,213.0,NaN,NaN,0.0,174.0,457661.0,...,4178.0,4904.0,0.0,0.0,2.0,2.0,0.0,35009.0,0.0,35009.0
3116,WY,2016,773.0,0.0,773.0,NaN,NaN,0.0,149.0,425087.0,...,3604.0,4356.0,0.0,0.0,2.0,2.0,0.0,40522.0,0.0,40522.0
3117,WY,2017,505.0,0.0,505.0,NaN,NaN,0.0,285.0,426707.0,...,4189.0,5045.0,0.0,0.0,0.0,0.0,0.0,39806.0,0.0,39806.0
3118,WY,2018,614.0,0.0,614.0,NaN,NaN,0.0,187.0,424234.0,...,4210.0,4929.0,0.0,0.0,0.0,0.0,0.0,36936.0,0.0,36936.0


### Calculate Variables

Our strategy is as follows: for each variable, if all of its sub-variables are already calculated in the dataset, then we calculate that variable. If not, we skip to the next variable.

In [14]:
# transform calc to series of value in dict type
sf_calc = df_calc.groupby(["id"]).apply(
    lambda x: 
    x[["Variable", "Coefficient"]]
    .set_index("Variable")
    .transpose()
    .loc["Coefficient"]
    .to_dict()
)
sf_calc

id
Biodiesel                                                         {'BDTCB': 1}
Biodiesel->Transportation                                         {'BDACB': 1}
BiodieselLoss                                                     {'BDLCB': 1}
BiodieselLoss->Industrial                                         {'BDLCB': 1}
BiodieselSum                              {'Biodiesel': 1, 'BiodieselLoss': 1}
                                                          ...                 
Wood->Residential                                                 {'WDRCB': 1}
WoodProduction                                                    {'WDPRB': 1}
WoodProduction->DensifiedBiomassExport                            {'WDEXB': 1}
WoodProduction->Wood                                              {'WDPRB': 1}
WoodWaste                                              {'Wood': 1, 'Waste': 1}
Length: 96, dtype: object

In [15]:
# calculate the variables

# Our strategy is as follows: for each variable, if all of its sub-variables are already calculated in the dataset, then we calculate that variable. If not, we skip to the next variable.

# number of variables still needed to calculte. By default it equals the number of variables to calculate
to_calculate = len(sf_calc)

while(to_calculate > 0):

    # a dictionary that holds all variables
    d_to_calculate = {}

    for var in sf_calc.index:
        if(var in df_data.columns): 
            sf_calc.drop(var)
            continue

        if(all(x in df_data.columns for x in sf_calc[var].keys())):
            d_to_calculate[var] = sf_calc[var]
            df_data[var] = 0

    to_calculate = len(d_to_calculate)
    
    if(to_calculate == 0): break

    for var in d_to_calculate:
        for subvar in d_to_calculate[var]:
            df_data[var] += df_data[subvar].fillna(0) * d_to_calculate[var][subvar]

df_data

MSN,StateCode,Year,BDACB,BDLCB,BDTCB,CCEXB,CCIMB,CLACB,CLCCB,CLEIB,...,WoodWaste,Biofuel,Biomass,Consumption,FossilFuel,NoncombustibleRenewable,Nonrenewable,Renewable,Total,Clean
0,AK,1960,0.0,0.0,0.0,NaN,NaN,86.0,496.0,914.0,...,3681.0,0.0,3681.0,62445.0,54633.0,3120.0,54633.0,6801.0,61434.0,6801.0
1,AK,1961,0.0,0.0,0.0,NaN,NaN,42.0,496.0,1127.0,...,4145.0,0.0,4145.0,73349.0,64860.0,3168.0,64860.0,7313.0,72173.0,7313.0
2,AK,1962,0.0,0.0,0.0,NaN,NaN,43.0,593.0,1373.0,...,4246.0,0.0,4246.0,80171.0,71359.0,3207.0,71359.0,7453.0,78812.0,7453.0
3,AK,1963,0.0,0.0,0.0,NaN,NaN,36.0,393.0,1482.0,...,4383.0,0.0,4383.0,82436.0,72995.0,3410.0,72995.0,7793.0,80788.0,7793.0
4,AK,1964,0.0,0.0,0.0,NaN,NaN,33.0,306.0,2279.0,...,4728.0,0.0,4728.0,86891.0,76964.0,3374.0,76964.0,8102.0,85066.0,8102.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3115,WY,2015,213.0,0.0,213.0,NaN,NaN,0.0,174.0,457661.0,...,4906.0,3690.0,8596.0,1202644.0,773533.0,43790.0,773533.0,52386.0,825919.0,52386.0
3116,WY,2016,773.0,0.0,773.0,NaN,NaN,0.0,149.0,425087.0,...,4358.0,3948.0,8306.0,1157005.0,744203.0,50205.0,744203.0,58511.0,802714.0,58511.0
3117,WY,2017,505.0,0.0,505.0,NaN,NaN,0.0,285.0,426707.0,...,5045.0,3537.0,8582.0,1186405.0,772548.0,50866.0,772548.0,59448.0,831996.0,59448.0
3118,WY,2018,614.0,0.0,614.0,NaN,NaN,0.0,187.0,424234.0,...,4929.0,3467.0,8396.0,1194080.0,792120.0,46549.0,792120.0,54945.0,847065.0,54945.0


In [16]:
# keep only our needed variables
df_data = df_data[["StateCode", "Year"] + df_info["id"].tolist()]
df_data

MSN,StateCode,Year,WoodProduction,Solar,Wind,Geothermal,Hydropower,Wood,Waste,Biodiesel,...,Wind->ElectricPower,Wind->Commercial,Wood->Residential,Wood->Industrial,Wood->ElectricPower,Wood->Commercial,WoodProduction->Wood,WoodProduction->DensifiedBiomassExport,NetInterstateImport->ElectricPower,ElectricPower->NetInterstateExport
0,AK,1960,3681.0,0.0,0.0,0.0,3120.0,3681.0,0.0,0.0,...,0.0,0.0,1806.0,1840.0,0.0,34.0,3681.0,0.0,0.0,0.0
1,AK,1961,4145.0,0.0,0.0,0.0,3168.0,4145.0,0.0,0.0,...,0.0,0.0,1823.0,2288.0,0.0,35.0,4145.0,0.0,0.0,0.0
2,AK,1962,4246.0,0.0,0.0,0.0,3207.0,4246.0,0.0,0.0,...,0.0,0.0,1703.0,2511.0,0.0,32.0,4246.0,0.0,0.0,0.0
3,AK,1963,4383.0,0.0,0.0,0.0,3410.0,4383.0,0.0,0.0,...,0.0,0.0,1651.0,2700.0,0.0,31.0,4383.0,0.0,0.0,0.0
4,AK,1964,4728.0,0.0,0.0,0.0,3374.0,4728.0,0.0,0.0,...,0.0,0.0,1703.0,2993.0,0.0,32.0,4728.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3115,WY,2015,4904.0,27.0,35009.0,663.0,8091.0,4904.0,2.0,213.0,...,35009.0,0.0,4178.0,113.0,0.0,612.0,4904.0,0.0,-318978.0,318978.0
3116,WY,2016,4356.0,33.0,40522.0,663.0,8987.0,4356.0,2.0,773.0,...,40522.0,0.0,3604.0,113.0,0.0,638.0,4356.0,0.0,-297805.0,297805.0
3117,WY,2017,5045.0,45.0,39806.0,663.0,10352.0,5045.0,0.0,505.0,...,39806.0,0.0,4189.0,88.0,0.0,769.0,5045.0,0.0,-297161.0,297161.0
3118,WY,2018,4929.0,67.0,36936.0,663.0,8883.0,4929.0,0.0,614.0,...,36936.0,0.0,4210.0,87.0,0.0,632.0,4929.0,0.0,-289472.0,289472.0


### Write the dataset to files

In [17]:
# write data to json
df_data.groupby("StateCode").apply(
    lambda x:
    x.set_index("Year", drop=False)
    .to_dict("index")
).to_json("data.json")

In [18]:
# write data to csv
df_data.to_csv("data.csv", index=False)

## Final processing

At this time, we have already finished building our needed dataset, and the dataset can be loaded directly into the web application to render the visualisation.
This section of the script filters the dataset and transform it into a format that can be loaded into the web easier.


In [19]:
# Transform data to needed formats
# We discovered that there were some error data from before 2000. For example, some records listed Nuclear value as negative
# so we decided to select only data from year 2000 to 2019 only

USSTATES_NEEDED_YEARS = range(2000, 2020)

df_usstates = df_data[df_data["Year"].isin(USSTATES_NEEDED_YEARS)]

df_usstates.to_csv("final/data.csv", index=False)

# write data to json
df_usstates.groupby("StateCode").apply(
    lambda x:
    x.set_index("Year", drop=False)
    .to_dict("index")
).to_json("final/data.json")

df_usstates

MSN,StateCode,Year,WoodProduction,Solar,Wind,Geothermal,Hydropower,Wood,Waste,Biodiesel,...,Wind->ElectricPower,Wind->Commercial,Wood->Residential,Wood->Industrial,Wood->ElectricPower,Wood->Commercial,WoodProduction->Wood,WoodProduction->DensifiedBiomassExport,NetInterstateImport->ElectricPower,ElectricPower->NetInterstateExport
40,AK,2000,1898.0,0.0,0.0,86.0,10220.0,1898.0,0.0,0.0,...,0.0,0.0,1540.0,101.0,0.0,258.0,1898.0,0.0,0.0,0.0
41,AK,2001,2973.0,0.0,10.0,86.0,13905.0,2973.0,0.0,7.0,...,10.0,0.0,2513.0,19.0,0.0,442.0,2973.0,0.0,0.0,0.0
42,AK,2002,3050.0,0.0,0.0,86.0,14643.0,3050.0,143.0,12.0,...,0.0,0.0,2550.0,11.0,36.0,453.0,3050.0,0.0,0.0,0.0
43,AK,2003,3194.0,0.0,0.0,89.0,16023.0,3194.0,67.0,10.0,...,0.0,0.0,2685.0,11.0,0.0,498.0,3194.0,0.0,0.0,0.0
44,AK,2004,3239.0,0.0,0.0,88.0,15004.0,3239.0,85.0,19.0,...,0.0,0.0,2752.0,12.0,0.0,475.0,3239.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3115,WY,2015,4904.0,27.0,35009.0,663.0,8091.0,4904.0,2.0,213.0,...,35009.0,0.0,4178.0,113.0,0.0,612.0,4904.0,0.0,-318978.0,318978.0
3116,WY,2016,4356.0,33.0,40522.0,663.0,8987.0,4356.0,2.0,773.0,...,40522.0,0.0,3604.0,113.0,0.0,638.0,4356.0,0.0,-297805.0,297805.0
3117,WY,2017,5045.0,45.0,39806.0,663.0,10352.0,5045.0,0.0,505.0,...,39806.0,0.0,4189.0,88.0,0.0,769.0,5045.0,0.0,-297161.0,297161.0
3118,WY,2018,4929.0,67.0,36936.0,663.0,8883.0,4929.0,0.0,614.0,...,36936.0,0.0,4210.0,87.0,0.0,632.0,4929.0,0.0,-289472.0,289472.0


In [20]:

# Transform data to needed formats

# energy flow

df_energyflow = df_usstates[df_usstates["Year"] == 2019]

# write data to csv
df_energyflow.to_csv("final/energyflow.csv")

# write data to json
df_energyflow.groupby("StateCode").apply(
    lambda x:
    x.set_index("Year", drop=False)
    .to_dict("index")
).to_json("final/energyflow.json")

df_energyflow


MSN,StateCode,Year,WoodProduction,Solar,Wind,Geothermal,Hydropower,Wood,Waste,Biodiesel,...,Wind->ElectricPower,Wind->Commercial,Wood->Residential,Wood->Industrial,Wood->ElectricPower,Wood->Commercial,WoodProduction->Wood,WoodProduction->DensifiedBiomassExport,NetInterstateImport->ElectricPower,ElectricPower->NetInterstateExport
59,AK,2019,6406.0,40.0,1270.0,186.0,14456.0,6406.0,386.0,858.0,...,1270.0,0.0,5536.0,70.0,0.0,801.0,6406.0,0.0,0.0,0.0
119,AL,2019,169734.0,3628.0,0.0,141.0,101563.0,164427.0,1577.0,5205.0,...,0.0,0.0,1978.0,162162.0,0.0,286.0,169734.0,5308.0,-423161.0,423161.0
179,AR,2019,80986.0,2256.0,0.0,808.0,36819.0,72153.0,4040.0,3996.0,...,0.0,0.0,6899.0,64256.0,0.0,998.0,80986.0,8833.0,-118859.0,118859.0
239,AZ,2019,10937.0,73265.0,4937.0,345.0,55251.0,10937.0,425.0,2208.0,...,4937.0,0.0,5459.0,1285.0,3403.0,790.0,10937.0,0.0,-303039.0,303039.0
299,CA,2019,98301.0,405668.0,122311.0,99329.0,341549.0,98301.0,40974.0,27005.0,...,122212.0,53.0,27045.0,24317.0,43026.0,3912.0,98301.0,0.0,692703.0,-692703.0
359,CO,2019,16940.0,16781.0,96640.0,759.0,16127.0,16940.0,1095.0,1532.0,...,96601.0,14.0,13285.0,467.0,1267.0,1922.0,16940.0,0.0,33467.0,-33467.0
419,CT,2019,14313.0,7546.0,109.0,21.0,3811.0,14313.0,9000.0,1057.0,...,109.0,0.0,6697.0,3491.0,3157.0,969.0,14313.0,0.0,-87183.0,87183.0
479,DC,2019,0.0,920.0,0.0,22.0,0.0,0.0,1060.0,28.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,114126.0,-114126.0
539,DE,2019,788.0,1628.0,47.0,430.0,0.0,788.0,716.0,166.0,...,0.0,47.0,668.0,23.0,0.0,97.0,788.0,0.0,73902.0,-73902.0
599,FL,2019,108139.0,68405.0,0.0,10056.0,1869.0,97075.0,58739.0,3365.0,...,0.0,0.0,302.0,86104.0,10625.0,44.0,108139.0,11065.0,129424.0,-129424.0


In [21]:

# Transform data to needed formats
# states

df_states = df_usstates[
    ["StateCode", "Year"] +
    ["Hydropower", "Solar", "Wind", "Geothermal", "Biomass",
     "Coal", "Petroleum", "NaturalGas", "Nuclear", "Renewable", "Total"]
].rename(columns={"StateCode": "code", "Year": "year"})

# fill nan values with 0
df_states.fillna(0, inplace=True)

d_states = {
    "data": df_states.groupby("code")
    .apply(lambda x: x.to_dict("records"))
    .to_frame("years")
    .reset_index()
    .to_dict("records")
}

pd.Series(d_states).to_json("final/states.json")

d_states


{'data': [{'code': 'AK',
   'years': [{'code': 'AK',
     'year': 2000,
     'Hydropower': 10220.0,
     'Solar': 0.0,
     'Wind': 0.0,
     'Geothermal': 86.0,
     'Biomass': 2069.0,
     'Coal': 16455.0,
     'Petroleum': 275255.0,
     'NaturalGas': 437972.0,
     'Nuclear': 0.0,
     'Renewable': 12375.0,
     'Total': 742057.0},
    {'code': 'AK',
     'year': 2001,
     'Hydropower': 13905.0,
     'Solar': 0.0,
     'Wind': 10.0,
     'Geothermal': 86.0,
     'Biomass': 3445.0,
     'Coal': 15911.0,
     'Petroleum': 289327.0,
     'NaturalGas': 413049.0,
     'Nuclear': 0.0,
     'Renewable': 17446.0,
     'Total': 735733.0},
    {'code': 'AK',
     'year': 2002,
     'Hydropower': 14643.0,
     'Solar': 0.0,
     'Wind': 0.0,
     'Geothermal': 86.0,
     'Biomass': 3540.0,
     'Coal': 16429.0,
     'Petroleum': 277389.0,
     'NaturalGas': 420808.0,
     'Nuclear': 0.0,
     'Renewable': 18269.0,
     'Total': 732895.0},
    {'code': 'AK',
     'year': 2003,
     'Hydropowe